In [1]:
import ee
import geemap
import os
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

In [3]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [80]:
#Cloud Mask
def maskS2sr(image):
  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = ee.Number(2).pow(10).int()
  cirrusBitMask = ee.Number(2).pow(11).int()
  # Get the pixel QA band.
  qa = image.select('QA60')
  # All flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  # Return the masked image, scaled to TOA reflectance, without the QA bands.
  return image.updateMask(mask) \
      .copyProperties(image, ["system:time_start"])


In [ ]:
# Read the forest fire inventory data to collect X and Y coordinated to create an area filter for each forest fire event


# Read data from the CSV file
data = pd.read_csv('Fires.csv')
data = data.dropna()
# Create a list to hold Point objects and buffer results
points = []
buffers = []

# Create Point objects and buffer them
for _, row in data.iterrows():
    lon, lat = row['X'], row['Y']
    point = Point(lon, lat)
    buffer = point.buffer(0.035, cap_style=3)
    points.append(point)
    buffers.append(buffer)

Reading the forest fire inventory to read the dates of the fire and search for images before and after the fire with the period of 20 days. Then cach image is masked and Difference Normalized burned ratio, NDVI and MNDWI will be calculated. After that each image is saved in the drive. 

In [ ]:
# Loop through each set of inputs
for idx, geometry in enumerate(buffers):
    prefire_end = pd.to_datetime(data.loc[idx, 'prefire_end'])
    postfire_start = pd.to_datetime(data.loc[idx, 'postfire_start'])
    
    # Calculate prefire_start and postfire_end
    prefire_start = prefire_end - pd.Timedelta(days=20)
    postfire_end = postfire_start + pd.Timedelta(days=20)
    
    # Convert prefire_start and postfire_end to strings in the required format
    prefire_start_str = prefire_start.strftime('%Y-%m-%d')
    postfire_end_str = postfire_end.strftime('%Y-%m-%d')
    
    # Geojson of the burned area
    geometry = ee.Geometry.Polygon(geometry.__geo_interface__['coordinates'])

    # pre-fire and post-fire dates
    prefire_start = prefire_start_str
    prefire_end = prefire_end.strftime('%Y-%m-%d')
    postfire_start = postfire_start.strftime('%Y-%m-%d')
    postfire_end = postfire_end_str

    # Selecting a Satellite Imagery
    ImCol = 'COPERNICUS/S2'
    pl = 'Sentinel-2'
    
    # Location
    area = ee.FeatureCollection(geometry)

    # Select imagery by time and location
    imagery = ee.ImageCollection(ImCol)

    # Imagery will be collected depending on the location, time frame, and cloud cover
    prefireImCol = ee.ImageCollection(imagery.filterDate(prefire_start, prefire_end).filterBounds(area))
    postfireImCol = ee.ImageCollection(imagery.filterDate(postfire_start, postfire_end).filterBounds(area))

    # Cloud Mask
    def maskS2sr(image):
        # Bits 10 and 11 are clouds and cirrus, respectively.
        cloudBitMask = ee.Number(2).pow(10).int()
        cirrusBitMask = ee.Number(2).pow(11).int()
        # Get the pixel QA band.
        qa = image.select('QA60')
        # All flags should be set to zero, indicating clear conditions.
        mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
            .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
        # Return the masked image, scaled to TOA reflectance, without the QA bands.
        return image.updateMask(mask) \
            .copyProperties(image, ["system:time_start"])

    # Apply Cloud Mask
    prefire_CM_ImCol = prefireImCol.map(maskS2sr)
    postfire_CM_ImCol = postfireImCol.map(maskS2sr)

    pre_mos = prefireImCol.mosaic().clip(area)
    post_mos = postfireImCol.mosaic().clip(area)

    pre_cm_mos = prefire_CM_ImCol.mosaic().clip(area)
    post_cm_mos = postfire_CM_ImCol.mosaic().clip(area)

    preNBR = pre_cm_mos.normalizedDifference(['B8', 'B12'])
    postNBR = post_cm_mos.normalizedDifference(['B8', 'B12'])
    dNBR_unscaled = preNBR.subtract(postNBR)
    dNBR = dNBR_unscaled.multiply(1000)
    preNDVI = pre_cm_mos.normalizedDifference(['B8', 'B4'])
    postNDVI = post_cm_mos.normalizedDifference(['B8', 'B4'])
    
    preMNDWI = pre_cm_mos.normalizedDifference(['B11', 'B3'])
    postMNDWI = post_cm_mos.normalizedDifference(['B11', 'B3'])

    Pre_image = pre_cm_mos.select(['B4', 'B3', 'B2'])
    Post_image = post_cm_mos.select(['B4', 'B3', 'B2']) 

    # Export the dNBR image
    
    geemap.ee_export_image_to_drive(
        dNBR, description=f'{idx}_dNBR_.tif',folder='larges', scale=10, region=geometry)
    geemap.ee_export_image_to_drive(
        preNDVI, description=f'{idx}_preNDVI.tif',folder='larges', scale=10, region=geometry)
    
    geemap.ee_export_image_to_drive(
        postNDVI, description=f'{idx}_postNDVI.tif',folder='larges', scale=10, region=geometry)
    
    geemap.ee_export_image_to_drive(
        preMNDWI, description=f'{idx}_preMNDWI.tif',folder='larges', scale=10, region=geometry)
    
    geemap.ee_export_image_to_drive(
        postMNDWI, description=f'{idx}_postMNDWI.tif',folder='larges', scale=10, region=geometry)
    geemap.ee_export_image_to_drive(
        Pre_image, description=f'{idx}_Pre_image.tif',folder='larges', scale=10, region=geometry)
    
    geemap.ee_export_image_to_drive(
        Post_image, description=f'{idx}_Post_image.tif',folder='larges', scale=10, region=geometry)

    # Clear the map for the next run
    Map.clear_layers()